Got 31.05 score. Used linear models with time as a feature to predict changes in quaternion-based coordinates (considering that "e" is supposed to be from 0 to 1; if prediction is incorrect, saved the original value of e_sim).

In [1]:
from coosys import cartesian_to_kepler as ctk, cartesian_to_quaternion as ctq
from coosys import kepler_to_cartesian as ktc, quaternion_to_cartesian as qtc
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import PolynomialFeatures as PF
from tqdm.auto import tqdm
tqdm.pandas()

from matplotlib import pyplot as plt
%matplotlib inline

D:\Anaconda3\envs\Standard\lib\site-packages\tqdm\std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
base = 1.3885 * 10**9

train_data = pd.read_csv("IDAO 2020/train.csv", encoding="utf8")
train_data["epoch"] = pd.to_datetime(train_data["epoch"]).apply(pd.Timestamp.timestamp) - base

test_data = pd.read_csv("IDAO 2020/Track 1/test.csv", encoding="utf8")
test_data["epoch"] = pd.to_datetime(test_data["epoch"]).apply(pd.Timestamp.timestamp) - base

In [3]:
gamma_km = 398603

def transform_row(row, func):
    return func(row.values)

def transform_dataset(data, func=lambda data: ctk(data, gamma_km), 
                      columns_from=("x", "y", "z", "Vx", "Vy", "Vz"), 
                      columns_to=("a", "e", "inclination", "longitude", "argument", "anomaly")):
    new_data = data.loc[:, list(columns_from)].progress_apply(transform_row, axis=1, result_type="expand", args=(func, ))
    new_data.columns = columns_to
    return data.drop(list(columns_from), axis=1).join(new_data)

In [4]:
train_quaternion = transform_dataset(train_data, func=lambda data: ctq(data, gamma_km),
                                     columns_from=("x", "y", "z", "Vx", "Vy", "Vz"),
                                     columns_to=(("a", "e", "anomaly", "q1", "q2", "q3", "q4")))
train_quaternion = transform_dataset(train_quaternion, func=lambda data: ctq(data, gamma_km), 
                                     columns_from=("x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"),
                                     columns_to=("a_sim", "e_sim", "anomaly_sim", "q1_sim", "q2_sim", "q3_sim", "q4_sim"))

In [5]:
def phase_regression(phases):
    periods = 0
    new_phases = phases.copy()
    for i in range(1, phases.size):
        if phases[i] < phases[i - 1]:
            periods += 1
        new_phases[i] += 2 * np.pi * periods
    return new_phases

def phase_degression(phases):
    new_phases = np.zeros_like(phases)
    for i in range(phases.size):
        new_phases[i] = phases[i] - 2 * np.pi * np.trunc(phases[i] / 2 / np.pi) 
    return new_phases

In [6]:
sats_to_predict = set(test_data["sat_id"].unique())

In [7]:
test_quaternion = transform_dataset(test_data, func=lambda data: ctq(data, gamma_km), 
                                    columns_from=("x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"),
                                    columns_to=("a_sim", "e_sim", "anomaly_sim", "q1_sim", "q2_sim", "q3_sim", "q4_sim"))

In [8]:
def predict_one(train_quaternion, test_quaternion, sat_id, degree=1):
    #data preparation
    train_sat = train_quaternion[train_quaternion.sat_id == sat_id]
    test_sat = test_quaternion[test_quaternion.sat_id == sat_id]
    result = pd.DataFrame(columns=["id", "a", "e", "anomaly", "q1", "q2", "q3", "q4"])
    result["id"] = test_sat["id"]
    train_t = train_sat["epoch"].to_numpy().reshape(-1, 1)
    train_features = PF(degree, include_bias=False).fit_transform(train_t)
    test_t = test_sat["epoch"].to_numpy().reshape(-1, 1)
    test_features = PF(degree, include_bias=False).fit_transform(test_t)
    
    #main prediction
    for coordinate in ["a", "e", "q1", "q2", "q3", "q4"]:
        train_diff = train_sat[coordinate] - train_sat[coordinate + "_sim"]
        model = LR().fit(train_features, train_diff)
        result[coordinate] = test_sat[coordinate + "_sim"] + model.predict(test_features)
        
    #anomaly prediction considers its periodicity
    train_diff = phase_regression(train_sat["anomaly"].to_numpy()) - phase_regression(train_sat["anomaly_sim"].to_numpy())
    model = LR().fit(train_features, train_diff)
    result["anomaly"] = phase_degression(test_sat["anomaly_sim"].to_numpy() + model.predict(test_features))
    
    #cheking correctness of e prediction
    if np.any(result["e"].to_numpy() >= 1.) or np.any(result["e"].to_numpy() < 0.):
        result["e"] = test_sat["e_sim"]

    return result

In [9]:
results = []

for sat_id in tqdm(sats_to_predict):
    results.append(predict_one(train_quaternion, test_quaternion, sat_id))
    
result_quaternion = pd.concat(results)

In [10]:
result_cartesian = transform_dataset(result_quaternion, func=lambda data: qtc(data, gamma_km), 
                                     columns_from=("a", "e", "anomaly", "q1", "q2", "q3", "q4"),
                                     columns_to=("x", "y", "z", "Vx", "Vy", "Vz"))

In [11]:
result_cartesian.to_csv("submission.csv", index=False)